<a href="https://colab.research.google.com/github/elianedp/ExpertSystem/blob/main/%5BONLINE%5D_Exemplo_2_Agente_de_di%C3%A1logo_baseado_em_regras_e_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemplo 2 - Agente de diálogo híbrido (baseado em regras e dados)
## Tecnólogo em Inteligência Artificial Aplicada - Agentes Conversacionais
Neste notebook iremos construir um agente de diálogo que trará ocorrências sobre determinado tema.

### Qual o contexto de nosso agente?

Iremos desenvolver um agente de diálogo de question answering, que baseado em um corpus de texto sobre um assunto, trará as informações mais relevantes de acordo com a consulta do usuário.

### Quais ferramentas e técnicas iremos utilizar?

*   **NLTK** - O mais famoso toolkit de Processamento de Linguagem Natural em Python.
*   **Expressões Regulares** - o pacote de regex do Python será utilizado para otimizar a busca de padrões.
*   **urllib e BeautifulSoup** - Bibliotecas para obter dados de páginas HTML.
*   **scikit-learn** - Pacote com funcionalidades de manipulaçã de dados e Machine Learning, vamos utilizar TF-IDF e Similaridade de cosseno.



### Construindo o agente de diálogo
Nosso agente vai operar da seguinte maneira:

1.   Recebe **entrada** do usuário
2.   **Pré-processa** a entrada do usuário
3.   Calcula a **similaridade** entre a entrada e as sentenças do corpus
4.   Obtém a sentença **mais similar do corpus**
5.   Mostra-a como **resposta** ao usuário

Anteriormente a estas etapas, iremos criar nosso corpus ao obter dados da Wikipedia, automaticamente, então vamos lá!


#### Importando as bibliotecas
Vamos importar o pacote de expressões regulares do Python e também o acesso ao WordNet dado pelo NLTK.

#### Construindo o corpus
Vamos fazer um *web-scraping* para obter os dados automaticamente da wikipedia. Este processo precisa ser executado apenas uma vez, e o arquivo salvo em forma de texto na máquina.

In [21]:
# Importando bibliotecas necessárias
import nltk
import numpy as np
import random
import string
import bs4 as bs
import urllib.request
import re
import warnings
from nltk.corpus import stopwords

# Download dos recursos do NLTK
nltk.download('punkt')      # tokenização
nltk.download('rslp')       # stemmer em português
nltk.download('stopwords')  # lista de stopwords

# Ignorar alguns avisos desnecessários
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##### Pré-processando o corpus
Como você pode ver no trecho acima, precisamos remover caracteres especiais do texto, além de dividí-lo em sentenças válidas.

In [22]:
# Aqui eu busquei todos os parágrafos do artigo
paragrafos = html_processado.find_all('p')

texto = ''

# Eu percorri os parágrafos e juntei os textos
for p in paragrafos:
    texto += p.text

# Aqui eu também peguei os dados da tabela lateral (infobox)
infobox = html_processado.find('table', {'class':'infobox'})
if infobox:
    for linha in infobox.find_all('tr'):
        dados = linha.get_text(" ", strip=True)
        texto += " " + dados

# Agora eu adicionei manualmente algumas frases curtas,
# pra garantir que o bot saiba responder perguntas básicas.
texto += " O presidente do Brasil é Luiz Inácio Lula da Silva. "
texto += " A população do Brasil é de aproximadamente 212 milhões de habitantes. "
texto += " O Brasil tem 26 estados e o Distrito Federal. "
texto += " O Brasil fica na América do Sul e faz fronteira com quase todos os países, exceto Chile e Equador. "

# Normalizei tudo para minúsculas
texto = texto.lower()

# Visualizando só o início pra conferir
texto[0:1000]

'\nbrasil (localmente\xa0[bɾaˈziw][c]), oficialmente república federativa do brasil (escutarⓘ),[12] é o maior país da américa do sul e da região da américa latina, sendo o quinto maior do mundo em área territorial (equivalente a 47,3% do território sul-americano), com cerca de 8,5 milhões de quilômetros quadrados,[5][13][14] e o sétimo em população[15][16] (com 212 milhões de habitantes, em julho de 2024).[17] é o único país na américa onde se fala majoritariamente a língua portuguesa e o maior país lusófono do planeta,[18] além de ser uma das nações mais multiculturais e etnicamente diversas, em decorrência da forte imigração oriunda de variados locais do mundo. sua atual constituição, promulgada em 1988, concebe o brasil como uma república federativa presidencialista,[12] formada pela união de 26 estados, do distrito federal e dos 5\xa0571 municípios.[12][19][nota 1]\nbanhado pelo oceano atlântico, o brasil tem um litoral de 7\xa0491\xa0km[18] e faz fronteira com todos os outros país

In [23]:
# Aqui eu estou limpando o texto, tirando as referências tipo [1], [2], etc.
texto = re.sub(r'\[[0-9]*\]', ' ', texto)

# Também deixo só um espaço onde tiver muitos espaços seguidos
texto = re.sub(r'\s+', ' ', texto)

# Aqui eu mostro um pedaço do texto só pra conferir
texto[0:1000]

' brasil (localmente [bɾaˈziw][c]), oficialmente república federativa do brasil (escutarⓘ), é o maior país da américa do sul e da região da américa latina, sendo o quinto maior do mundo em área territorial (equivalente a 47,3% do território sul-americano), com cerca de 8,5 milhões de quilômetros quadrados, e o sétimo em população (com 212 milhões de habitantes, em julho de 2024). é o único país na américa onde se fala majoritariamente a língua portuguesa e o maior país lusófono do planeta, além de ser uma das nações mais multiculturais e etnicamente diversas, em decorrência da forte imigração oriunda de variados locais do mundo. sua atual constituição, promulgada em 1988, concebe o brasil como uma república federativa presidencialista, formada pela união de 26 estados, do distrito federal e dos 5 571 municípios. [nota 1] banhado pelo oceano atlântico, o brasil tem um litoral de 7 491 km e faz fronteira com todos os outros países sul-americanos, exceto chile e equador, sendo limitado a 

In [24]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [25]:
# Aqui eu quebrei o texto em sentenças
sentencas = nltk.sent_tokenize(texto, language='portuguese')

# Também quebrei em palavras
palavras = nltk.word_tokenize(texto, language='portuguese')

# Mostro algumas sentenças só pra conferir
sentencas[10:15]


['em 1815, o brasil se torna parte de um reino unido com portugal.',
 'dom pedro i, o primeiro imperador, proclamou a independência política do país em 1822. inicialmente independente como um império, período no qual foi uma monarquia constitucional parlamentarista, o brasil tornou-se uma república em 1889, em razão de um golpe militar chefiado pelo marechal deodoro da fonseca (o primeiro presidente), embora uma legislatura bicameral, agora chamada de congresso nacional, já existisse desde a ratificação da primeira constituição, em 1824. desde o início do período republicano, a governança democrática foi interrompida por longos períodos de regimes autoritários, até um governo civil e eleito democraticamente assumir o poder em 1985, com o fim da ditadura militar.',
 'como potência regional e média, a nação tem reconhecimento e influência internacional, sendo que também é classificada como uma potência global emergente e como uma potencial superpotência por vários analistas.',
 'o pib no

#### Funções de pré-processamento de entrada do usuário
Vamos criar funções para pré-processar as entradas do usuário, vamos retirar pontuações e usar Stemming nos textos, para que palavras similares sejam processadas de maneira igual pelo algoritmo (e.g., pedra e pedregulho teriam a mesma forma léxica)

In [26]:
!pip install unidecode

In [27]:
# Aqui eu criei uma função que faz o Stemming, ou seja,
# ela pega as palavras e reduz para a raiz delas.
def stemming(tokens):
    stemmer = nltk.stem.RSLPStemmer()
    novo_texto = []
    for token in tokens:
        novo_texto.append(stemmer.stem(token.lower()))
    return novo_texto

# Aqui eu monto um dicionário para remover pontuações
removePontuacao = dict((ord(punctuation), None) for punctuation in string.punctuation)

# Essa função faz o pré-processamento completo:
# - coloca em minúsculas
# - tira acento
# - remove pontuação
# - tira stopwords
# - aplica stemming
import unidecode
stop_words = set(stopwords.words("portuguese"))

def preprocessa(documento):
    # normalizo: minúsculas e sem acento
    doc = unidecode.unidecode(documento.lower())
    # tiro a pontuação
    doc = doc.translate(removePontuacao)
    # quebro em tokens
    tokens = nltk.word_tokenize(doc, language="portuguese")
    # tiro stopwords e aplico stemming
    tokens = [n for n in stemming(tokens) if n not in stop_words]
    return tokens

# Testando como fica depois do pré-processamento
preprocessa("Olá meu nome é Lucas, eu moro no Brasil, e você?")

['ola', 'nom', 'luc', 'mor', 'brasil', 'voc']

#### Resposta à saudações
Mesmo que estejamos construindo um sistema de diálogo (baseado em tarefas), é normal que o usuário inicie conversas com saudações ao agente. Portanto, iremos desenvolver rapidamente uma função (i.e., regras) para lidar especialmente com esta situação.
Vamos criar algumas respostas possíveis, e sempre vamos escolher aleatóriamente uma delas, para evitar que nosso agente fique repetitivo.

In [28]:
# Aqui eu defini algumas formas que o usuário pode usar para cumprimentar o bot
saudacoes_entrada = ("olá", "bom dia", "boa tarde", "boa noite", "oi", "como vai", "e aí")

# E aqui eu fiz uma listinha de respostas possíveis do bot
saudacoes_respostas = [
    "olá",
    "olá, espero que esteja tudo bem contigo",
    "oi",
    "oie",
    "seja bem-vindo, em que posso te ajudar?"
]

# Essa função verifica se a entrada do usuário foi uma saudação
# Se for, eu retorno uma resposta aleatória da lista acima
def geradorsaudacoes(saudacao):
    for token in saudacao.split():
        if token.lower() in saudacoes_entrada:
            return random.choice(saudacoes_respostas)

# Testando várias vezes eu vou ver respostas diferentes
geradorsaudacoes("Olá")


'oie'

#### Resposta à consultas do usuário
Agora, teremos uma função para lidar com consultas do usuário. Onde iremos comparar a similaridade entre a entrada do usuário, com as sentenças do corpus. Caso encontremos, a sentença mais similar será mostrada como resposta.

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Essa função vai gerar a resposta do bot
def geradorrespostas(entradausuario):
    resposta = ''

    # Eu adiciono a pergunta do usuário junto com as sentenças do corpus
    sentencas.append(entradausuario)

    # Aqui eu crio a matriz TF-IDF usando minha função de pré-processamento
    word_vectorizer = TfidfVectorizer(tokenizer=preprocessa, stop_words=stopwords.words('portuguese'))
    all_word_vectors = word_vectorizer.fit_transform(sentencas)

    # Calculo a similaridade de cosseno entre a entrada e as sentenças
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Pego a sentença mais parecida (tirando a última, que é a própria pergunta)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    # Se a similaridade for zero ou muito baixa, eu aviso que não encontrei nada
    if vector_matched < 0.3:  # aqui eu coloquei um limite de confiança
        resposta = "Me desculpe, não encontrei nada relevante sobre isso."
    else:
        resposta = sentencas[similar_sentence_number]

    # Eu removo a pergunta do usuário da lista de sentenças para não poluir o corpus
    sentencas.pop(-1)

    return resposta

#### Interagindo com o agente de diálogo
Vamos definir um algoritmo que continue interagindo com o usuário até que ele decida finalizar.

O resultado não é sempre o ideal, mas já cobre muitas possíveis perguntas. Se utilizássemos apenas regras de diálogo para responder perguntas sobre um tema, precisaríamos de centenas de regras para tal. Mas como baseamos nossas respostas em dados apenas uma regra que calcula similaridade com nosso corpus já é o suficiente.

Faça perguntas como:
*  *Qual o esporte mais popular no Brasil?*
*  *Quais eventos esportivos o Brasil já organizou?*
*  *Como é a cozinha brasileira?*
*  *Onde são realizadas pesquisas tecnológicas no Brasil?*


In [ ]:
continue_dialogue = True
print("Olá, eu sou o Agente Tupiniquim. Me pergunte qualquer coisa sobre o Brasil.")

while continue_dialogue:
    # Aqui eu pego a entrada do usuário
    human_text = input("Você: ").lower()

    # Se o usuário disser tchau, eu encerro
    if human_text == "tchau":
        continue_dialogue = False
        print("Agente Tupiniquim: Até a próxima.")

    # Se for agradecimento, eu também encerro
    elif human_text in ["obrigado", "muito obrigado", "agradecido"]:
        continue_dialogue = False
        print("Agente Tupiniquim: Disponha")

    # Se for uma saudação, eu respondo de forma aleatória
    elif geradorsaudacoes(human_text) is not None:
        print("Agente Tupiniquim:", geradorsaudacoes(human_text))

    # Caso contrário, eu tento responder usando o corpus
    else:
        print("Agente Tupiniquim:", geradorrespostas(human_text))

Olá, eu sou o Agente Tupiniquim. Me pergunte qualquer coisa sobre o Brasil.
Agente Tupiniquim: isto criou uma cozinha nacional marcada pela preservação das diferenças regionais.
Agente Tupiniquim: o futebol é o esporte mais popular no brasil.
Agente Tupiniquim: Me desculpe, não encontrei nada relevante sobre isso.
Agente Tupiniquim: Me desculpe, não encontrei nada relevante sobre isso.


### O que pode ser feito?
Utilizamos um modelo baseado em regras, onde uma das regras faz uso de um corpus de dados para formular as respostas, o que já deixou nosso modelo bem mais flexível, sem necessidade de criação de centenas/milhares de regras.

Mas, o que poderíamos fazer para melhorar?


*   Poderíamos obter não apenas os parágrafos(`<p>`) na página da wikipedia, mas também utilizar os dados dispostos na coluna direita, que apresentam informações bem relevantes como população, atual presidente, etc., para montar sentenças.
*   Poderíamos melhorar ainda mais o cálculo de similaridade ao utilizar um modelo de Word Embeddings, além do TF-IDF.
*   Obter dados sobre o Brasil de diferentes fontes.
*   Criar um classificador de contexto para o agente, e de maneira dinâmica buscar páginas da Wikipedia correspondentes à pergunta do usuário, e só então dar a resposta. Assim nosso agente não ficaria limitado a perguntas sobre o Brasil.



## Referências e Material complementar

* [Python for NLP: Creating a Rule-Based Chatbot](https://stackabuse.com/python-for-nlp-creating-a-rule-based-chatbot/)
* [Building a Simple Chatbot from Scratch in Python (using NLTK)](https://morioh.com/p/6cc33336784c)
* [Building a simple chatbot in python](https://medium.com/nxtplus/building-a-simple-chatbot-in-python-3963618c490a)
* [Designing A ChatBot Using Python: A Modified Approach](https://towardsdatascience.com/designing-a-chatbot-using-python-a-modified-approach-96f09fd89c6d)
* [Build Your First Python Chatbot Project](https://dzone.com/articles/python-chatbot-project-build-your-first-python-pro)
* [Python Chatbot Project – Learn to build your first chatbot using NLTK & Keras](https://data-flair.training/blogs/python-chatbot-project/)
* [Python Chat Bot Tutorial - Chatbot with Deep Learning (Part 1)](https://www.youtube.com/watch?v=wypVcNIH6D4)
* [Intelligent AI Chatbot in Python](https://www.youtube.com/watch?v=1lwddP0KUEg)
* [Coding a Jarvis AI Using Python 3 For Beginners](https://www.youtube.com/watch?v=NZMTWBpLUa4)

Este notebook foi produzido por Prof. [Lucas Oliveira](http://lattes.cnpq.br/3611246009892500).